In [1]:
# expand the cell width to 100% of the content
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Logistische Gleichung

Die logistische Gleichung ist eine relativ einfache, nichtlineare Gleichung, die sich im Gegensatz zu den uns bisher bekannten linearen Gleichungssystemen für bestimmte Parameter scheinbar unvorhersehbar und chaotisch verhält.  
Sie ist rekursiv definiert durch die folgende Gleichung:  
<p style="text-align: center; font-size: 20px;">
<b> x<sub>n+1</sub> = r * x<sub>n</sub> * (1 - x<sub>n</sub>) </b> </p>  
    
## Populationsmodellierung durch exponentielles Wachstum
Die logistische Gleichung wurde von Pierre-François Verhulst, einem belgischen Mathematiker, in seiner 1845 erschienenen Arbeit "..." zur Modellierung von Populationen entwickelt.  
Ein Ansatz dazu ist, die Population durch eine exponentiell wachsende Funktion x<sub>n+1</sub> = r * x<sub>n</sub> zu modellieren.  
x<sub>n</sub> ist dabei die Größe der Population zum Zeitpunkt n, r eine Spezies-spezifische Wachstumsrate (i.d.R. > 1, sonst wäre die Spezies schon ausgestorben).  

Dies lässt sich zu Beginn so auch in der Realität beobachten, beispielsweise bei der Vermehrung von Zellen oder dem Wachstum von Pilzen.  
Auch für Tiere und Menschen scheint dies intuitiv: Je dichter ein gegebener Lebensraum mit einer Spezies besiedelt ist, desto höher ist auch die Wahrscheinlichkeit, dass sich zwei paarungswillige und -fähige Individuen treffen und sich vermehren.  

Allerdings ist auch offensichtlich, dass diese Modellierung nicht die gesamte Entwicklung beschreiben kann:  
Die Funktion wächst exponentiell und wird somit irgendwann sehr schnell unendlich groß, wohingegen die Population einer Spezies in der Realität begrenzt ist. Irgendwann reichen beispielsweise die vorhandenen Ressourcen im Lebensraum (Platz, Nahrung) nicht mehr für alle aus, wodurch die Sterblichkeit steigt und so die effektive Wachstumsrate (Geburtenrate - Sterberate) abfällt. Die Population wächst dann langsamer oder schrumpft sogar. Diese äußeren Einflüsse fasst man in der <i> Kapazität K</i> eines Lebensraums zusammen. Sie beschreibt die Größe einer Population, bei der ihre Wachstumsrate 0 beträgt. Sie ist dann stabil, das heißt es sterben genau so viele Individuen wie geboren werden.

## Ergänzung durch wachstumshemmende Terme
Verhulst erkannte dieses Problem und löste es, indem er neben dem wachstumstreibenden Term x<sub>n</sub> noch einen wachstumshemmenden in die Gleichung einbaute.  
Beschreibt die Kapazität K wie oben definiert die maximale Individuenanzahl, so muss die Population umso langsamer wachsen, je näher sich ihre Größe schon an die Kapazität angenähert hat.  
Dies erreicht man zum Beispiel durch das Hinzufügen des Faktors (K - x<sub>n</sub>) ⇔ (1 - x<sub>n</sub> / K). Dadurch entsteht die logistische Gleichung x<sub>n+1</sub> = r * x<sub>n</sub> * (1 - x<sub>n</sub> / K).  
Häufig wird das K einfach weggelassen bzw. mit 1 gleichgesetzt, wodurch x die Populationsgröße nicht mehr absolut, sondern relativ zur Kapazität beschreibt.  
Auch die nachfolgende Simulation betrachtet die Gleichung wie zu Beginn notiert ohne Betrachtung einer spezifischen Kapazität.
In ihr erkennt man gut, dass sich die Populationsgröße für 1 < r < 3 irgendwann stabilisiert.  
Für r = 1 ∨ r > 3 schwingt die Gleichung jedoch oder wird sogar - wie eingangs erwähnt - scheinbar unvorhersehbar chaotisch.


In [14]:
class log_equation:
    def __init__(self, r):
        self.r = r
    
    def f(self, x):
        return self.r * x * (1 - x)

@interact
def plot_le(x0 = slider(0, 1, default=0.5, step_size=0.1, label="Startwert x<sub>0</sub>"), 
            r  = slider(0, 4, default=3.8, step_size=0.1, label="Wachstum r"),
            darstellung = selector(['Punkte', 'Kurve'], label="Darstellung", default='Kurve')):
    le = log_equation(r)
    x = x0
    p = [(0, x)]
    for i in range(1, 100):
        x = le.f(x)
        p += [(i, x)]
    show(line(p) if darstellung == 'Kurve' else points(p), figsize=8, ymin=0, ymax=1)
    


SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIHBsb3RfbGUgYXQgMHg2ZmZiYzhiZWUxNDA+IHdpdGggMyB3aWRnZXRzCiAgeDA6IFRyYW5zZm9ybUZsb2F0U2xpZGVyKHZhbHVlPTAuNSzigKY=


# Feigenbaum Diagramm


# ToDo: - Feigenbaum-Konstante-Erklärung - Feigenbaum Aduktoren

Das Feigenbaumdiagramm ist ein nach dem US-amerikanischen Physiker Mitchell Feigenbaum benanntes Diagramm, welches nichtlineare Systeme, wie z.B. die eben beschriebene logistische Gleichung, visualisiert.

Im Folgenden sieht man eine Implementierung für das Feigenbaumdiagramm der logistischen Gleichung. <br>
Zur Berechnung werden die spezifizierte Nummer an Punkten mit zufälligen x- und r-Werten im Gesammten Bereich generiert. Danach wird die logistische Gleichung N-mal auf jeden Punkt andewandt und die Punkte anschließend in der Form (r,x<sub>N</sub>) geplottet. <br>
Über die Regler lassen sich einige Werte, wie die Anzahl der Punkte, Iterationen pro Punkt oder der betrachtete R-Bereich individuell einstellen.<br>
Den R<sub>Min</sub>-Wert größer als R<sub>Max</sub> zu wählen ist logisch unsinnig und erzeugt demnach kein Diagramm.

Um kleinere Änderungen Vorzunehmen können sie einen Schieber eines Reglers selektieren und mit den Pfeiltasten anpassen.
<br>Um größere Änderungen Vorzunehmen klicken sie bitte auf den Regler an der gewünschten Stelle, anstatt den Schieber zu verschieben.
<br>Bei einer hohen Zahl an Punkten oder Iterationen kann es etwas dauern, bis das eingestellte Diagramm erstellt wurde.

In [15]:
@interact
def plot_fig(NumPoints = slider(1, 100000, default=500, step_size=1, label="Num<sub>Points</sub>"),
             N = slider(0, 100, default=5, step_size=1, label="N<sub>Iterations</sub>"),
             rMax = slider(1.05, 4, default=4, step_size=0.1, label="R<sub>Max</sub>"),
             rMin = slider(0.00, 4, default=1, step_size=0.1, label="R<sub>Min</sub>"),
             dotSize = slider(1, 50, default=1, step_size=1, label="Size<sub>Dot</sub>"),
             auto_update = false):
    
    p = []
    for i in range(0, NumPoints):
        p += [(random()*rMax,random())]
    #p = list(filter(lambda x: x[0] > rMin, p))
    for i in range(len(p)):
        for n in range(N):
            #Aus performance-technischen Gründen wird hier direkt die Formel und nicht die Funktion von oben benutzt
            # Punkt = (r, xN)
            p[i] = (p[i][0], p[i][0] * p[i][1] * (1 - p[i][1]))
            
    #print(len(ps))
    show(points(p, size=dotSize), figsize=8, ymin=0, ymax=1, xmin=0, xmax=rMax)

TWFudWFsIGludGVyYWN0aXZlIGZ1bmN0aW9uIDxmdW5jdGlvbiBwbG90X2ZpZyBhdCAweDZmZmJkNDVhZTFiOD4gd2l0aCA1IHdpZGdldHMKICBOdW1Qb2ludHM6IFRyYW5zZm9ybUludFNsaWTigKY=


# Sensitivität




Die Sensitivität eines Systems ist grundlegend die Abhägigkeit dessen von seinen Anfangsparametern. Bei sensitiven Systemen genügen sehr kleine Abweichungen von diesem Anfangszustand um völlig unterschiedliche Verläufe auszulösen.

Ist möglich Regeln zu definieren um den Verlauf zu beschreiben, so spricht man von deterministischem Chaos. Auch bei chaotisch anmutenden Begebenheiten liegen dem deterministischen Chaos immer Regeln zugrunde, mit denen sich das sensitive System beschreiben und voraussagen ließe, würde man es komplett überblicken.

Im Gegenzug dazu gibt es noch vorgänge, welche sich nicht klar durch Regeln beschreiben lassen. Im Fall von quantenmechanischen Prozessen können keine Regelun gefunden werden, da nach heutigem Stand der Wissenschaft quanten sich echt unvorhersehbar verhalten. Wenn ein System nun auf diese quantenmechanischen Prozesse hin sensitiv ist, so lässt sich dessen Verlauf in keinem Fall vorhersagen.

Die bereits beschriebene Logistische Gleichung ist ein Beispiel für eine anfangsbedingungs-sensitive Gleichung.

## Schmetterlingseffekt

Der Schmetterlingseffekt ist wohl der bekanntesten Effekte dem die Sensitivität der Anfangsbedingungen zu Grunde liegt. 
Der Metereologe Edward Norton Lorenz entdeckte das Phänomen 1961, als er Berechnungen zu einem Wettermodell durchführte. Hierbei setzte er verschiedene Wetterparameter un Verbindung und ließ sein Modell mehrere Male berechnen, wobei er jedoch völlig unterschiedliche Ergebnisse erhielt. Dies geschah, weil er bei der erneuten Berechnung Zwischenergebnisse der Ersten verwendete, welche gerundet und weniger genau waren. 



# Julia-Mengen

Klassische Formel: <br />
    Zn+1 = Zn^2 + c <br />
c ist eine komplexe konstante. <br />
z ist eine Zahl auf der komplexen Ebene. <br />
n ist die Genauigkeit.

Zu jedem Punkt einer Mandelbrotmenge gibt es eine zugehoerige Julia-Menge, wobei dieser eben die Konstante C ist.

# Mandelbrotmenge

Aehnlich zur Julia-Menge wird die Mandelbrotmenge berechnet. Dabei haben die Variablen der Formel jedoch eine andere Bedeutung:
Z0 beginnt bei jeder komplexen Zahl mit 0.
c ist eine Zahl auf der komplexen Ebene. <br />
z ist das Ergebnis der letzten rekursion auf dieser komplexen Zahl. <br />
n ist die Genauigkeit.

Die Mandelbrotmenge ist der Bereich, bei welchem Z nach n iterationen nicht den Radius g ueberschreitet und somit zu 0 konvergiert.

# Apfelmännchen und Selbstähnlichkeit

Apfelmaenchen ist eine visuelle Darstellung der Mandelbrotmenge, wobei die X Achse auf dem Koordinatensystem der reele und die Y Achse der imaginaere Teil der Komplexen Zahl ist.

Zeichnen:
Um die Menge zu zeichnen muss man ueber jeden Pixel iterieren und auf diesen die gegebene Formel n mal anwenden. Falls der Wert ueber den Radius g (Bei der klassischen Mandelbrotmenge 2) schreitet, wird anhand der benoetigten Iterationen die Farbe festgestellt. Wenn dieser Schwellwert nach n iterationen nicht ueberschritten wird, wird das Pixel schwarz gefaerbt.

Mit anpassen der Formel, beispielsweise zu Z^3 + c, ergeben sich andere Fraktale mit eigenen Eigenschaften.

# Quellen
Logistische Gleichung:

- Die logistische Gleichung - ein Weg ins Chaos (Andreas Schmid, 2009, http://www.fraktalwelt.de/systeme/as_weg_ins_chaos.pdf)
- Modellierung natürlicher Prozesse und Optimierungsstrategien (Jens Kortus, Theoretische Physik TU Freiberg (https://tu-freiberg.de/sites/default/files/media/institut-fuer-theoretische-physik-10451/Lehre/Modelierung/modnatproz.pdf)
- Populationsmodelle (Philipp Jansche, 2008/09, https://www.mathi.uni-heidelberg.de/~thaeter/mathmod08/AusarbeitungPopulationsmodelle.pdf)
- Die logistische Gleichung als ein Beispiel für chaotische Prozesse in der Physik (Nils Grzech, 2012, https://docplayer.org/18693722-Die-logistische-gleichung-als-ein-beispiel-fuer-chaotische-prozesse-in-der-physik.html)

Feigenbaum-Diagramm:

- Feigenbaum-Konstante https://physik.cosmos-indirekt.de/Physik-Schule/Feigenbaum-Konstante
- Diagramm erzeugen http://walter.bislins.ch/blog/index.asp?page=Feigenbaum%2DDiagramm+erzeugen+und+analysieren

Sensitivität:

- Chaosforschung https://physik.cosmos-indirekt.de/Physik-Schule/Chaosforschung
- Schmetterlingseffekt:
    - https://www.br.de/wissen/edward-lorenz-meteorologe-schmetterlingseffekt-chaostheorie-100.html 
    - https://physik.cosmos-indirekt.de/Physik-Schule/Schmetterlingseffekt
    
The dark side of the Mandelbrot set / Mathologer: <br />
https://www.youtube.com/watch?v=9gk_8mQuerg

Die Mandelbrotmenge / Uni-Leipzig: <br />
https://www.informatik.uni-leipzig.de/~meiler/Schuelerseiten.dir/DPlotzki/html/mndlbrt.htm